# Cayla Mason
# PH700A Assignment 2

In [1]:
library(tidyverse)
library(magrittr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘magrittr’


The following object is masked from ‘package:purrr’:

    set_names


The following object is masked from ‘package:tidyr’:

    extract




In [2]:
# read in aggregated data
vcf.agg <- read.csv("class_vcf_aggregated-1.csv", header = TRUE, sep = '\t')
head(vcf.agg, n=3)

,position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,total_mut_res,inh,lineages
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>
1,17654,T,G,missense_variant,pknA,Rv0015c,1109,370,Q/P,160,23,S,3
2,17657,T,C,missense_variant,pknA,Rv0015c,1106,369,Q/R,159,23,S,3
3,23811,T,TTGC,upstream_gene_variant,fhaB,Rv0019c,,,,11,9,S,2


In [3]:
# change data types
vcf.agg$gene.position <- as.numeric(vcf.agg$gene.position)
vcf.agg$protein.position <- as.numeric(vcf.agg$protein.position)

cols <- c("consequence","gene","inh")
vcf.agg %<>% mutate(across(cols, as.factor))

head(vcf.agg)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(cols)` instead of `cols` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.



,position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,total_mut_res,inh,lineages
,<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<fct>,<int>
1,17654,T,G,missense_variant,pknA,Rv0015c,1109,370,Q/P,160,23,S,3
2,17657,T,C,missense_variant,pknA,Rv0015c,1106,369,Q/R,159,23,S,3
3,23811,T,TTGC,upstream_gene_variant,fhaB,Rv0019c,NA,NA,,11,9,S,2
4,24698,G,C,synonymous_variant,fhaA,Rv0020c,747,249,G,219,94,S,4
5,24704,T,C,synonymous_variant,fhaA,Rv0020c,741,247,Q,327,138,S,5
6,24707,C,G,missense_variant,fhaA,Rv0020c,738,246,E/D,339,145,S,5


In [4]:
# read in unaggregated data
vcf.unagg <- read.csv("class-vcf-parsed-unaggregated.csv", header = TRUE, sep = '\t')
head(vcf.unagg, n=3)

,position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,variant.type,isolate,lineage,inh
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,17654,T,G,missense_variant,pknA,Rv0015c,1109,370,Q/P,SNV,1047-07,East-Asian,S
2,17657,T,C,missense_variant,pknA,Rv0015c,1106,369,Q/R,SNV,1047-07,East-Asian,S
3,23811,T,TTGC,upstream_gene_variant,fhaB,Rv0019c,,,,insertion,1047-07,East-Asian,S


In [5]:
# change data types
vcf.unagg$gene.position <- as.numeric(vcf.unagg$gene.position)
vcf.unagg$protein.position <- as.numeric(vcf.unagg$protein.position)

cols <- c("consequence","gene","variant.type","isolate","lineage","inh")
vcf.unagg %<>% mutate(across(cols, as.factor))

head(vcf.unagg)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


,position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,variant.type,isolate,lineage,inh
,<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<fct>,<fct>,<fct>,<fct>
1,17654,T,G,missense_variant,pknA,Rv0015c,1109,370,Q/P,SNV,1047-07,East-Asian,S
2,17657,T,C,missense_variant,pknA,Rv0015c,1106,369,Q/R,SNV,1047-07,East-Asian,S
3,23811,T,TTGC,upstream_gene_variant,fhaB,Rv0019c,NA,NA,,insertion,1047-07,East-Asian,S
4,24698,G,C,synonymous_variant,fhaA,Rv0020c,747,249,G,SNV,1047-07,East-Asian,S
5,24704,T,C,synonymous_variant,fhaA,Rv0020c,741,247,Q,SNV,1047-07,East-Asian,S
6,24707,C,G,missense_variant,fhaA,Rv0020c,738,246,E/D,SNV,1047-07,East-Asian,S


## Question 3a: identify the individual isolates that contain the following variants:
- rpoB codons 426-452<br>
rpoB codons should be denoted by protein.position where gene==rpoB
- inhA promoter: C-15T, A-16G, T-8C, T-8A <br>
inhA promoter amino acid changes should be denoted by genomic position, "The starting position of inhA is 1674202. So for problem #3, the inhA promoter mutations should be searched for with respect to that start site; e.g. C-15T would occur at position 1674187"
- katG S315T<br>
gene==katG, protein.position==315.

In [24]:
# use unaggregated data to identify isolates
vcf.unagg %>% 
    filter((gene == "rpoB" & between(protein.position, 426, 452)) |   # any changes in rpoB specified codons
           (gene == "katG" & protein.position == 315 & AA == "S/T") | # katG S315T
           (between(position,1674169, 1674172) & AA == "T/C") |       # inhA promoter T8C
           (between(position,1674169, 1674172) & AA == "T/A") |       # inhA promoter T8A
           (between(position,1674187, 1674189) & AA == "C/T") |       # inhA promoter C15T
           (between(position,1674190, 1674192) & AA == "A/G")         # inhA promoter A16G          
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

[1] 209

## Question 3b: for each mutation, how many are INH-R and INH-S?

In [16]:
# look at the data
# sort the aggregated vcf file according to each mutation
vcf.agg %<>% arrange(desc(gene), AA, inh)
head(vcf.agg)

,position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,total_mut_res,inh,lineages
,<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<fct>,<int>
1,1626655,T,C,missense_variant,zwf2,Rv1447c,308,103,E/G,45,41,R,2
2,1626655,T,C,missense_variant,zwf2,Rv1447c,308,103,E/G,45,4,S,1
3,1626656,C,T,missense_variant,zwf2,Rv1447c,307,103,E/K,5,2,R,1
4,1626656,C,T,missense_variant,zwf2,Rv1447c,307,103,E/K,5,3,S,1
5,1626022,T,C,missense_variant,zwf2,Rv1447c,941,314,Q/R,9,2,R,1
6,1626022,T,C,missense_variant,zwf2,Rv1447c,941,314,Q/R,9,7,S,1


In [23]:
total.mut <- vcf.agg %>% 
    filter(inh == "R") %>% 
    nrow()

total.mut

[1] 9769

In [25]:
# assuming a threshold of 50%
# at least 50% of isolates contain a given mutation and confer resistance
vcf.agg %>% 
    filter(inh == "R",
           (total_mut_res/total_mut >= 0.5)) %>% 
    nrow()

[1] 4359

## Question 4
For each variant the two diagnostic assays in #3, calculate performance measures. Calculate them again for each of the diagnostic performs in aggregate (aggregated over all mutations assayed by the diagnostic). Of these two which would you choose as the programmatic molecular DST method? Why?<br>
- sensitivity = TP / (TP + FN)
- specificity = TN / (TN + FP)
- positive predicted value = TP + FP
- negative predicted value = FN + TN

In [32]:
# how many total isolates are there?
vcf.unagg %>% select(isolate) %>% unique() %>% nrow()

[1] 499

In [33]:
# how many are actually resistant?
vcf.unagg %>% filter(inh == "R") %>% select(isolate) %>% unique() %>% nrow()

[1] 249

In [34]:
# how many are actually susceptible?
vcf.unagg %>% filter(inh == "S") %>% select(isolate) %>% unique() %>% nrow()

[1] 250

In [31]:
# rpoB codons 426-452
# of the isolates with a mutation, how many were truly INH resistant and INH susceptible?
vcf.agg %>% 
    filter((gene == "rpoB" & between(protein.position, 426, 452)))     

position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,total_mut_res,inh,lineages
<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<fct>,<int>
761139,C,T,missense_variant,rpoB,Rv0667,1333,445,H/Y,6,6,R,2
761161,T,C,missense_variant,rpoB,Rv0667,1355,452,L/P,5,5,R,2
761155,C,T,missense_variant,rpoB,Rv0667,1349,450,S/L,144,139,R,4
761155,C,T,missense_variant,rpoB,Rv0667,1349,450,S/L,144,5,S,2


In [35]:
# katG S315T
vcf.agg %>% 
    filter(gene == "katG" & protein.position == 315 & AA == "S/T")

position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,total_mut_res,inh,lineages
<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<fct>,<int>
2155168,C,G,missense_variant,katG,Rv1908c,944,315,S/T,193,190,R,4
2155168,C,G,missense_variant,katG,Rv1908c,944,315,S/T,193,3,S,2


In [36]:
# inhA promoter T8C
vcf.agg %>% 
    filter(between(position,1674169, 1674172) & AA == "T/C")

position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,total_mut_res,inh,lineages
<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<fct>,<int>


In [37]:
# inhA promoter T8A
vcf.agg %>% 
    filter(between(position,1674169, 1674172) & AA == "T/A")

position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,total_mut_res,inh,lineages
<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<fct>,<int>


In [38]:
# inhA promoter C15T
vcf.agg %>% 
    filter(between(position,1674187, 1674189) & AA == "C/T")  

position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,total_mut_res,inh,lineages
<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<fct>,<int>


In [39]:
# inhA promoter A16G 
vcf.agg %>% 
    filter(between(position,1674190, 1674192) & AA == "A/G")

position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,total_mut_res,inh,lineages
<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<fct>,<int>


In [40]:
# GeneXPert is all rpoB codons 426-452
# total isolates with a GeneXPert hit
vcf.unagg %>% 
    filter(gene == "rpoB" & between(protein.position, 426, 452)) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

[1] 155

In [41]:
# GeneXPert
# resistant isolates with a GeneXPert hit
vcf.unagg %>% 
    filter(gene == "rpoB" & between(protein.position, 426, 452),
           inh == "R"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

[1] 150

In [42]:
# GenoType MTBDRplus VER 2.0
# total isolates with hit
vcf.unagg %>% 
    filter(
           (gene == "katG" & protein.position == 315 & AA == "S/T") | # katG S315T
           (between(position,1674169, 1674172) & AA == "T/C") |       # inhA promoter T8C
           (between(position,1674169, 1674172) & AA == "T/A") |       # inhA promoter T8A
           (between(position,1674187, 1674189) & AA == "C/T") |       # inhA promoter C15T
           (between(position,1674190, 1674192) & AA == "A/G")         # inhA promoter A16G          
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

[1] 193

In [43]:
# GenoType MTBDRplus VER 2.0
# resistant isolates with hit
vcf.unagg %>% 
    filter(
           ((gene == "katG" & protein.position == 315 & AA == "S/T") | # katG S315T
           (between(position,1674169, 1674172) & AA == "T/C") |       # inhA promoter T8C
           (between(position,1674169, 1674172) & AA == "T/A") |       # inhA promoter T8A
           (between(position,1674187, 1674189) & AA == "C/T") |       # inhA promoter C15T
           (between(position,1674190, 1674192) & AA == "A/G")),        # inhA promoter A16G 
            inh == "R"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

[1] 190

## Question 5
Now, it is your mission to use the whole dataset to identify supplementary or alternative variants to include an improve diagnostic for INH resistance.
(A) What mutations would you include to capture 90% of resistant cases (e.g. sensitivity)? 95% of resistant cases? In going from 90 to 95% sensitivity, what performance are you sacrificing?
(B) Imagining yourself as the Infectious Disease Molecular Diagnostics consultant for a country’s government, which would you recommend? Why?

I think the best approach is to maximize both sensitivity and specificity equally. Upon reduction to a manageable number of variants: sort by sensitivity in descending order. Add one variant at a time to the test and assess sensitivity of the test.

In [44]:
vcf.agg %<>% arrange(desc(total_mut), desc(total_mut_res))
head(vcf.agg)

,position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,total_mut_res,inh,lineages
,<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<fct>,<int>
1,1728837,A,G,upstream_gene_variant,pks5,Rv1527c,NA,NA,,490,248,R,6
2,3247883,T,C,synonymous_variant,ppsA,Rv2931,2439,813,S,490,246,R,6
3,3896340,T,G,missense_variant,,Rv3479,521,174,L/R,490,246,S,5
4,3247883,T,C,synonymous_variant,ppsA,Rv2931,2439,813,S,490,244,S,5
5,3896340,T,G,missense_variant,,Rv3479,521,174,L/R,490,244,R,6
6,1728837,A,G,upstream_gene_variant,pks5,Rv1527c,NA,NA,,490,242,S,5


I do not think it makes any sense that synonymous variants would lead to resistance. 
I think it is more likely that resistant isolates  happen to have synonymous variants.
I will remove those.

In [91]:
vcf.agg.filt <- vcf.agg %>% 
    filter(consequence != "synonymous_variant")

vcf.agg.filt %<>% arrange(desc(total_mut), desc(total_mut_res))
head(vcf.agg.filt)

,position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,total_mut_res,inh,lineages
,<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<fct>,<int>
1,1728837,A,G,upstream_gene_variant,pks5,Rv1527c,NA,NA,,490,248,R,6
2,3896340,T,G,missense_variant,,Rv3479,521,174,L/R,490,246,S,5
3,3896340,T,G,missense_variant,,Rv3479,521,174,L/R,490,244,R,6
4,1728837,A,G,upstream_gene_variant,pks5,Rv1527c,NA,NA,,490,242,S,5
5,2415656,G,C,missense_variant,murD,Rv2155c,739,247,R/G,489,246,R,6
6,2752698,C,A,upstream_gene_variant,rpfE,Rv2450c,NA,NA,,489,245,S,5


In [92]:
vcf.agg.filt %<>% 
    filter(inh == "R") %>% 
    mutate(
        TP = total_mut_res, 
        FP = total_mut - TP,
        TN = 250 - FP,
        FN = 249 - TP,
        sensitivity = (TP*100)/(TP+FN),
        specificity = (TN*100)/(TN+FP)
    )

head(vcf.agg.filt)

,position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,total_mut_res,inh,lineages,TP,FP,TN,FN,sensitivity,specificity
,<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,1728837,A,G,upstream_gene_variant,pks5,Rv1527c,NA,NA,,490,248,R,6,248,242,8,1,99.59839,3.2
2,3896340,T,G,missense_variant,,Rv3479,521,174,L/R,490,244,R,6,244,246,4,5,97.99197,1.6
3,2415656,G,C,missense_variant,murD,Rv2155c,739,247,R/G,489,246,R,6,246,243,7,3,98.79518,2.8
4,2752698,C,A,upstream_gene_variant,rpfE,Rv2450c,NA,NA,,489,244,R,6,244,245,5,5,97.99197,2.0
5,1536251,G,T,missense_variant,,Rv1364c,1394,465,A/E,489,244,R,6,244,245,5,5,97.99197,2.0
6,2612632,C,A,missense_variant,,Rv2337c,356,119,G/V,488,245,R,6,245,243,7,4,98.39357,2.8


In [93]:
vcf.agg.filt %<>% 
    mutate(variant = paste(gene,protein.position, AA))

head(vcf.agg.filt)

,position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,total_mut_res,inh,lineages,TP,FP,TN,FN,sensitivity,specificity,variant
,<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1728837,A,G,upstream_gene_variant,pks5,Rv1527c,NA,NA,,490,248,R,6,248,242,8,1,99.59839,3.2,pks5 NA
2,3896340,T,G,missense_variant,,Rv3479,521,174,L/R,490,244,R,6,244,246,4,5,97.99197,1.6,174 L/R
3,2415656,G,C,missense_variant,murD,Rv2155c,739,247,R/G,489,246,R,6,246,243,7,3,98.79518,2.8,murD 247 R/G
4,2752698,C,A,upstream_gene_variant,rpfE,Rv2450c,NA,NA,,489,244,R,6,244,245,5,5,97.99197,2.0,rpfE NA
5,1536251,G,T,missense_variant,,Rv1364c,1394,465,A/E,489,244,R,6,244,245,5,5,97.99197,2.0,465 A/E
6,2612632,C,A,missense_variant,,Rv2337c,356,119,G/V,488,245,R,6,245,243,7,4,98.39357,2.8,119 G/V


In [94]:
vcf.agg.filt2 <- vcf.agg.filt %>% 
    filter(sensitivity >= 65,
           specificity >= 65
          ) %>% 
    arrange(desc(sensitivity))

vcf.agg.filt2

position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,total_mut_res,inh,lineages,TP,FP,TN,FN,sensitivity,specificity,variant
<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,<int>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
2155168,C,G,missense_variant,katG,Rv1908c,944,315,S/T,193,190,R,4,190,3,247,59,76.30522,98.8,katG 315 S/T
2338811,A,G,missense_variant,,Rv2082,103,35,K/E,268,182,R,4,182,86,164,67,73.09237,65.6,35 K/E
104962,G,A,missense_variant,,Rv0095c,254,85,A/V,241,167,R,5,167,74,176,82,67.06827,70.4,85 A/V
2975227,T,G,missense_variant,,Rv2650c,8,3,N/T,247,164,R,5,164,83,167,85,65.86345,66.8,3 N/T
2867298,C,A,missense_variant,lppB,Rv2544,175,59,H/N,230,164,R,5,164,66,184,85,65.86345,73.6,lppB 59 H/N
4254290,T,G,missense_variant,,Rv3798,1298,433,L/R,212,164,R,4,164,48,202,85,65.86345,80.8,433 L/R
105007,C,G,missense_variant,,Rv0095c,209,70,S/T,208,164,R,6,164,44,206,85,65.86345,82.4,70 S/T
4120926,A,G,missense_variant,,Rv3680,1132,378,N/D,237,163,R,4,163,74,176,86,65.46185,70.4,378 N/D
75233,C,A,upstream_gene_variant,,Rv0067c,NA,NA,,243,162,R,4,162,81,169,87,65.06024,67.6,NA


In [97]:
vcf.unagg %<>% mutate(variant = paste(gene,protein.position, AA))

In [98]:
total_mut_res <- vcf.unagg %>% 
    filter(inh == "R",
          variant == "katG 315 S/T" | # katG S315T
          variant == "35 K/E"
          ) %>%  
    select(isolate) %>% 
    unique() %>% 
    nrow()

sens = total_mut_res/249
sens

[1] 0.7630522

In [99]:
total_mut_res <- vcf.unagg %>% 
    filter(inh == "R",
           variant == "katG 315 S/T" | # katG S315T
           variant == "85 A/V"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

sens = total_mut_res/249
sens

[1] 0.7630522

In [100]:
total_mut_res <- vcf.unagg %>% 
    filter(inh == "R",
           variant == "katG 315 S/T" | # katG S315T
           variant == "70 S/T"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

sens = total_mut_res/249
sens

[1] 0.7630522

In [101]:
total_mut_res <- vcf.unagg %>% 
    filter(inh == "R",
           variant == "katG 315 S/T" | # katG S315T
           variant == "433 L/R"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

sens = total_mut_res/249
sens

[1] 0.7630522

In [102]:
total_mut_res <- vcf.unagg %>% 
    filter(inh == "R",
           variant == "katG 315 S/T" | # katG S315T
           variant == "lppB 59 H/N"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

sens = total_mut_res/249
sens

[1] 0.8875502

In [103]:
total_mut_res <- vcf.unagg %>% 
    filter(inh == "R",
           variant == "katG 315 S/T" | # katG S315T
           variant == "3 N/T"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

sens = total_mut_res/249
sens

[1] 0.7630522

In [104]:
total_mut_res <- vcf.unagg %>% 
    filter(inh == "R",
           variant == "katG 315 S/T" | # katG S315T
           variant == "378 N/D"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

sens = total_mut_res/249
sens

[1] 0.7630522

In [105]:
total_mut_res <- vcf.unagg %>% 
    filter(inh == "R",
           variant == "katG 315 S/T" | # katG S315T
           variant == "PPE3 257 E/K"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

sens = total_mut_res/249
sens

[1] 0.8313253

In [106]:
total_mut_res <- vcf.unagg %>% 
    filter(inh == "R",
           variant == "katG 315 S/T" | # katG S315T
           variant == "PE_PGRS18 NA"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

sens = total_mut_res/249
sens

[1] 0.7630522

In [107]:
# combine katG S315T, lppB H59N, PPE3 E257K
total_mut_res <- vcf.unagg %>% 
    filter(inh == "R",
           variant == "katG 315 S/T" | # katG S315T
           variant == "PPE3 257 E/K" |
           variant == "lppB 59 H/N"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

sens = total_mut_res/249
sens

[1] 0.9156627

In [108]:
vcf.agg.filt %<>% mutate(SS.average = (sensitivity + specificity)/2) %>% 
    arrange(desc(SS.average))

head(vcf.agg.filt)

,position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,⋯,inh,lineages,TP,FP,TN,FN,sensitivity,specificity,variant,SS.average
,<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,⋯,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,2155168,C,G,missense_variant,katG,Rv1908c,944,315,S/T,193,⋯,R,4,190,3,247,59,76.30522,98.8,katG 315 S/T,87.55261
2,761155,C,T,missense_variant,rpoB,Rv0667,1349,450,S/L,144,⋯,R,4,139,5,245,110,55.82329,98.0,rpoB 450 S/L,76.91165
3,105007,C,G,missense_variant,,Rv0095c,209,70,S/T,208,⋯,R,6,164,44,206,85,65.86345,82.4,70 S/T,74.13173
4,1151304,C,A,missense_variant,kdpD,Rv1028c,383,128,R/L,147,⋯,R,3,133,14,236,116,53.41365,94.4,kdpD 128 R/L,73.90683
5,484005,G,T,missense_variant,fadD30,Rv0404,29,10,R/L,147,⋯,R,3,133,14,236,116,53.41365,94.4,fadD30 10 R/L,73.90683
6,1446733,G,A,missense_variant,argS,Rv1292,355,119,A/T,147,⋯,R,3,133,14,236,116,53.41365,94.4,argS 119 A/T,73.90683


In [109]:
total_mut_res <- vcf.unagg %>% 
    mutate(variant = paste(gene,protein.position, AA)) %>% 
    filter(inh == "R",
           variant == "katG 315 S/T" | 
           variant == "PPE3 257 E/K" |
           variant == "lppB 59 H/N"  |
           variant == "rpoB 450 S/L"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

sens = total_mut_res/249
sens

[1] 0.9196787

In [110]:
vcf.agg.filt %<>% arrange(desc(sensitivity))

head(vcf.agg.filt)

,position,REF,ALT,consequence,gene,Rv.number,gene.position,protein.position,AA,total_mut,⋯,inh,lineages,TP,FP,TN,FN,sensitivity,specificity,variant,SS.average
,<int>,<chr>,<chr>,<fct>,<fct>,<chr>,<dbl>,<dbl>,<chr>,<int>,⋯,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>
1,2165286,A,C,missense_variant,PPE34,Rv1917c,2026,676,S/A,484,⋯,R,5,248,236,14,1,99.59839,5.6,PPE34 676 S/A,52.59920
2,1728837,A,G,upstream_gene_variant,pks5,Rv1527c,NA,NA,,490,⋯,R,6,248,242,8,1,99.59839,3.2,pks5 NA,51.39920
3,2415656,G,C,missense_variant,murD,Rv2155c,739,247,R/G,489,⋯,R,6,246,243,7,3,98.79518,2.8,murD 247 R/G,50.79759
4,3746409,A,G,missense_variant,PPE55,Rv3347c,6776,2259,L/P,481,⋯,R,6,245,236,14,4,98.39357,5.6,PPE55 2259 L/P,51.99679
5,968426,A,AGCCGGGTTG,inframe_insertion,PE_PGRS15,Rv0872c,NA,NA,-/QPG,486,⋯,R,6,245,241,9,4,98.39357,3.6,PE_PGRS15 NA -/QPG,50.99679
6,4338595,GC,G,upstream_gene_variant,whiB6,Rv3862c,NA,NA,,487,⋯,R,6,245,242,8,4,98.39357,3.2,whiB6 NA,50.79679


In [115]:
total_mut_res <- vcf.unagg %>% 
    mutate(variant = paste(gene,protein.position, AA)) %>% 
    filter(inh == "R",
           variant == "katG 315 S/T" | 
           variant == "PPE3 257 E/K" |
           variant == "lppB 59 H/N"  #|
           #variant == "PPE34 676 S/A"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

sens = total_mut_res/249
sens

[1] 0.9156627

all 4 variants 0.987951807228916<br>
-lpp 0.983935742971888<br>
-ppe3 0.975903614457831<br>
-ppe34 0.91566265060241<br>
-> keep katG, ppe3, and ppe34

In [121]:
total_mut <- vcf.unagg %>% 
    mutate(variant = paste(gene,protein.position, AA)) %>% 
    filter(variant == "katG 315 S/T" | 
           variant == "PPE3 257 E/K" |
           #variant == "lppB 59 H/N"  |
           variant == "PPE34 676 S/A"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

total_mut_res <- vcf.unagg %>% 
    mutate(variant = paste(gene,protein.position, AA)) %>% 
    filter(inh == "R",
           variant == "katG 315 S/T" | 
           variant == "PPE3 257 E/K" |
           #variant == "lppB 59 H/N"  |
           variant == "PPE34 676 S/A"
          ) %>% 
    select(isolate) %>% 
    unique() %>% 
    nrow()

sens = total_mut_res/249
sens

spec = (250 - (total_mut - total_mut_res))/250
spec

[1] 0.9839357

[1] 0.104